**Objective:** Based on the names and aliases of various companies, find multiple mentions of these companies in news articles and attempt to:

1. Create a graph of words/people/topics/associated companies (and try to determine if the term/person is good/bad for the company)

    - Try to analyze the source of the relationship (e.g., Público: 3 mentions, TVI: 10 mentions, ...)

2. Explore the relationship between news (positive/negative) and stock prices.

3. ...

Trabalho tem de ter 3 partes:

1. project structure + data acquisition

2. exploratory data analysis and visualization

3. results & discussion

Fonte de Dados: arquivo.pt (https://github.com/arquivo/pwa-technologies/wiki/Arquivo.pt-API)

Ideias semelhantes:

- https://github.com/politiquices

- https://github.com/msramalho/desarquivo

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime
import re
from joblib import load

---

# data01.parquet

**sites dos quais vamos obter as noticias**

In [11]:
# news from https://www.kadaza.pt

def news(csvFile = 'noticias.csv'):
    """
    grab the news websites from a csv file
    """
    links = pd.read_csv(csvFile, header=None).iloc[:,0]
    return ",".join(links)

news()

'www.publico.pt,publico.pt,www.dn.pt,www.rtp.pt,rpt.pt,www.cmjornal.pt,www.iol.pt,www.tvi24.iol.pt,tvi24.iol.pt,noticias.sapo.pt,observador.pt,expresso.pt,www.expresso.pt,sol.sapo.pt,www.jornaldenegocios.pt,www.jn.pt,jn.pt,ionline.pt,sicnoticias.pt,www.sicnoticias.pt,www.lux.iol.pt,www.ionline.pt,news.google.pt,www.dinheirovivo.pt,www.aeiou.pt,aeiou.pt,www.tsf.pt,tsf.pt,www.sabado.pt,dnoticias.pt,www.dnoticias.pt,economico.sapo.pt,cnnportugal.iol.pt'

**como vão ser os api requests / decidir as empresas (PSI20) a analisar / fazer api requests in 3years groups**

*1 year to 3 years is long enough to smooth out short-term fluctuations and identify underlying trends. Charts with weekly or monthly intervals over these periods show developments over full economic/market cycles.*

In [7]:
def api_request(search, websites, date):
    """
    search: expression/word (what to look for)
    websites: comma separated websites (where to look for)
    date: list such as [20030101, 20031231] (when to look for)
    -
    returns the responde_items from arquivo.pt api
    """
    search = f"q=%22{search.replace(' ', '%20')}%22"
    websites = f"&siteSearch={websites}"
    date = f"&from={date[0]}&to={date[1]}"    
    url = (
        f"https://arquivo.pt/textsearch?{search}{websites}{date}"
        "&fields=linkToArchive,linkToExtractedText,tstamp"
        "&maxItems=500&dedupValue=25&dedupField=url&prettyPrint=false&type=html"
        )
    json = requests.get(url).json()
    data = json["response_items"]
    if len(data) == 500:
        print(f"You might have lost some data: {search, date}")
    return data

In [ ]:
def datav1(companies):
    """
    this is the function where we choose the companies which will be in study
    -
    companies should be a dictionary
        {"company1": [aliases or other names the company is or was known by],
        "company2": [...]}
    -
    this data will be saved into a parquet file for future use and with already api requests

    also this will do the api requests .... get this better
    """
    # CREATING DF WITH COMPANIES AND THEIR ALIASES
    companies_data = {"companies": [], "aliases": []}
    for company in companies.keys():
        companies_data["companies"].append(company)
        companies_data["aliases"].append(companies[company])
    df = pd.DataFrame(companies_data).set_index("companies")

    # SITES OF WHERE TO LOOK FOR NEWS
    websites = news()

    # INITIALIZAING API REQUESTS
    # groups of 3 years, from 2000 to 2020
    for cluster in range(2000, 2021, 3):
        api_cluster = [] #reset api_cluster for each cluster (group of 3 year)
        print(f"Processing cluster: {cluster}")
        print("Processing company:", end=" ")
        # iterate over each company
        for company_aliases in df["aliases"]:
            api_company = [] #reset api_company for each company
            print(f"{company_aliases[0]}", end = "; ")
            # iterate over each company's aliases
            for alias in company_aliases:
                # iterate over each cluter's year
                for year in range(cluster, cluster + 3):                        
                    api_aliasS1 = api_request(alias, websites, [int(f"{year}0101"), int(f"{year}0630")])
                    api_aliasS2 = api_request(alias, websites, [int(f"{year}0701"), int(f"{year}1231")])
                    api_company += api_aliasS1 + api_aliasS2
            # save company data
            api_cluster.append(api_company)

        # save cluster (group of 3 years) data
        df[f"api.{cluster}"] = api_cluster
        print(f"{cluster} OK.")

    # save all data
    df.to_parquet("data01.parquet")
    print("Finished.")
    return df

companies = {"Banco Comercial Português": ["Banco Comercial Português", "BCP"],
             "Galp Energia": ["Galp Energia", "GALP"],
             "EDP": ["EDP", "Energias de Portugal", "Electricidade de Portugal"],
             "Sonae": ["Sonae", "SON"],
             "Mota-Engil": ["Mota-Engil", "EGL"]}
df01 = datav1(companies)
#df01

In [2]:
#df01 = pd.read_parquet("data01.parquet")
df01

,aliases,api.2000,api.2003,api.2006,api.2009,api.2012,api.2015,api.2018
companies,,,,,,,,
Banco Comercial Português,"[Banco Comercial Português, BCP]",[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...
Galp Energia,"[Galp Energia, GALP]",[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...
EDP,"[EDP, Energias de Portugal, Electricidade de P...",[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...
Sonae,"[Sonae, SON]",[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...
Mota-Engil,"[Mota-Engil, EGL]",[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...,[{'linkToArchive': 'https://arquivo.pt/wayback...


In [6]:
df01["api.2000"].loc["Mota-Engil"]

array([{'linkToArchive': 'https://arquivo.pt/wayback/20010725052410/http://www.dn.pt/neg/pna/23s3aa.htm', 'linkToExtractedText': 'https://arquivo.pt/textextracted?m=http%3A%2F%2Fwww.dn.pt%2Fneg%2Fpna%2F23s3aa.htm%2F20010725052410', 'tstamp': '20010725052410'},
       {'linkToArchive': 'https://arquivo.pt/wayback/20010920124822/http://www.dn.pt/eco/20p37a.htm', 'linkToExtractedText': 'https://arquivo.pt/textextracted?m=http%3A%2F%2Fwww.dn.pt%2Feco%2F20p37a.htm%2F20010920124822', 'tstamp': '20010920124822'},
       {'linkToArchive': 'https://arquivo.pt/wayback/20010921000521/http://www.dn.pt/eco/20p37a.htm', 'linkToExtractedText': 'https://arquivo.pt/textextracted?m=http%3A%2F%2Fwww.dn.pt%2Feco%2F20p37a.htm%2F20010921000521', 'tstamp': '20010921000521'},
       {'linkToArchive': 'https://arquivo.pt/wayback/20020628024219/http://www.rtp.pt/teletexto/text/293-07.htm', 'linkToExtractedText': 'https://arquivo.pt/textextracted?m=http%3A%2F%2Fwww.rtp.pt%2Fteletexto%2Ftext%2F293-07.htm%2F200206

In [6]:
df01.map(lambda x: len(x))

,aliases,api.2000,api.2003,api.2006,api.2009,api.2012,api.2015,api.2018
companies,,,,,,,,
Banco Comercial Português,2,153,241,183,561,1074,1430,954
Galp Energia,2,128,389,272,582,1156,1391,968
EDP,3,133,339,173,653,1232,1970,1096
Sonae,2,192,435,279,502,1215,1705,1196
Mota-Engil,2,4,83,60,195,538,828,560


--- 

# data02.parquet

por ter usado `&dedupValue=25&dedupField=url` e diferentes aliases, há informação repetida

**filtrar repetidos e textos que não mencionem nenhum alias**

problemas ultrapassados:

- API has the following usage limits (250req/min, error 429): `time.sleep(60)`

- API error 404 for some urls: return 0 (False) and skip it

- extrair o texto demora muito: filtrar e salvar coluna a coluna

nota: podia ter feito já online o processamento do texto, mas não queria estar dependente do wifi

In [ ]:
def extracText(linkToExtractedText):
    # Infinite loop to handle retry logic in case of 429 Too Many Requests
    while True:
        response = requests.get(linkToExtractedText)
        status_code = response.status_code
        
        if status_code == 200:
            # If the request is successful (200 OK), return the extracted text
            soup = BeautifulSoup(response.content, "html.parser")
            return soup.get_text()
        elif status_code == 429:
            # Handle 429 Too Many Requests by reading the Retry-After header
            print(" (...)", end = "")
            time.sleep(60)  # Pause execution for the retry period
        elif status_code == 404:
            return 0
        else:
            # For any other status codes (e.g., 500, ...), print the status and break the loop
            print(f"Request failed with status code {status_code}. Link was {linkToExtractedText}")
            break

# Function to process each column
def filterColumn(column, aliases):
    """aliases in text, repeated text and extract text"""
    global stats
    filtered_column = []

    for row in aliases.index:
        filtered_cell = []
        seen_text = set()
        print(f"; {row}", end = "")
        for i in column.loc[row]:
            
            # Extract text from 'linkToExtractedText'
            text = extracText(i['linkToExtractedText'])
                

            # Skip if the text has already been processed
            if text in seen_text:
                stats["duplicate"] += 1
                continue

            elif not text: #ERROR 404
                stats["404"] += 1
                continue
            
            # Check if any alias is found in the text
            elif any(alias.lower() in text.lower() for alias in aliases.loc[row]):
                i["ExtractedText"] = text  # Add extracted text to the record
                
                # Remove unwanted fields
                i.pop('linkToExtractedText', None)
                
                # Append the processed record
                filtered_cell.append(i)
                
                # Mark this text as processed
                seen_text.add(text)

        filtered_column.append(filtered_cell)
                
    return filtered_column


def processColumns(col_to_proc):
    print(f"Starting: {datetime.now()}")
    try:
        # continuar df criada
        df = pd.read_parquet("data02.parquet")
    except:
       # criar df para trabalhar
       df = pd.read_parquet("data01.parquet").to_parquet("data02.parquet")
       df = pd.read_parquet("data02.parquet")
    for column in col_to_proc:
        has_link = "linkToExtractedText" in df.iloc[-1][column][-1]
        has_extracText = "ExtractedText" in df.iloc[-1][column][-1]
        if not has_link and has_extracText:
            print(f"\n{column} already done. Skipping.")
        else:
            print(f"\nProcessing {column}", end = ": ")
            df[column] = filterColumn(df[column], df["aliases"])
            df.to_parquet("data02.parquet")
    print(f"\nEnded: {datetime.now()}.")

stats = {"404": 0, "duplicate": 0}
processColumns(["api.2000", "api.2003", "api.2006", "api.2009", "api.2012", "api.2015", "api.2018"])
print(stats)

In [8]:
pd.read_parquet("data02.parquet").map(lambda x: len(x))

,aliases,api.2000,api.2003,api.2006,api.2009,api.2012,api.2015,api.2018
companies,,,,,,,,
Banco Comercial Português,2,90,191,169,497,983,1219,824
Galp Energia,2,66,233,181,469,964,1104,812
EDP,3,80,245,140,538,1076,1528,872
Sonae,2,130,318,239,459,1109,1400,1026
Mota-Engil,2,3,67,26,164,384,596,445


In [7]:
pd.read_parquet("data02.parquet").map(lambda x: len(x)) - pd.read_parquet("data01.parquet").map(lambda x: len(x))

,aliases,api.2000,api.2003,api.2006,api.2009,api.2012,api.2015,api.2018
companies,,,,,,,,
Banco Comercial Português,0,-63,-50,-14,-64,-91,-211,-130
Galp Energia,0,-62,-156,-91,-113,-192,-287,-156
EDP,0,-53,-94,-33,-115,-156,-442,-224
Sonae,0,-62,-117,-40,-43,-106,-305,-170
Mota-Engil,0,-1,-16,-34,-31,-154,-232,-115


---

# dtree01.csv & data03.parquet

used a decision tree to remove non news (ads or unrelated news to each company)

also used percentagens intervals in order to keep more news:

- if newsProb in [.0, .4[ - trash

- if newsProb in [.4, .6] - filter setences: keep only the ones with any alias

- if newsProb in ].6, 1.] - keep everything


In [2]:
# FEATURES
# [['IstALIAS', 'propAN', 'txtSZ', 'countALI', 'countDTS', 'countHOUR', 'countCAPS']]

# added "aliases" to all func in order to run them all without worrying about the inputs

def IstALIAS(text, aliases):
    """where does the first alias appear, title?"""
    indexs = []
    for alias in aliases:
        index = text.lower().find(alias.lower())
        if index != -1:
            indexs.append(index)
    try:
        a = text[:min(indexs)].count(' ')
    except:
        a = 10000000000000000000
    return a

def propAN(text, aliases):
    """proportion of alphanumeric chars in the text"""
    alphanumeric_chars = sum(char.isalnum() for char in text)
    proportion = alphanumeric_chars / len(text)
    return proportion

def txtSZ(text, aliases):
    """text size"""
    return len(text)

def countALI(text, aliases):
    """count how many aliases appear in the text"""
    alias_count = {expression: 0 for expression in aliases}
    for alias in aliases:
        # Use re.escape to handle any special characters in the expression
        pattern = re.escape(alias.lower())
        matches = re.findall(alias, text.lower())
        alias_count[alias] = len(matches)
    return sum(alias_count.values())

def countDTS(text, aliases):
    """count how many dates appear in the text"""
    date_pattern = r'\b(\d{1,2}[-/]\d{1,2}[-/]\d{2,4}|\d{4}[-/]\d{1,2}[-/]\d{1,2})\b'
    # 10/11/2024', '10/10/2024', '12-25-1990', '2024-11-05', '01/10/2024'
    dates = re.findall(date_pattern, text)
    date_count = len(dates)
    return date_count

def countHOUR(text, aliases):
    """count how many hours (ex.: hh:mm) appear in the text"""
    time_pattern = r'\b([01]?[0-9]|2[0-3]):[0-5][0-9]\b'   
    occurrences = re.findall(time_pattern, text)
    return len(occurrences)

def countCAPS(text, aliases):
    """count how many WORDS are upper"""
    words = text.split()
    uppercase_word_count = sum(1 for word in words if word.isupper())
    return uppercase_word_count

# Load the saved model
clf = load('dtree01.joblib')

In [4]:
def filter_sentences_by_keywords(text, aliases):
    # Split the text by punctuation and also by multiple spaces or newlines
    sentences = re.split(r'(?<=[.!?]) +|\s{2,}|\n+', text)
    # Filter sentences that contain any of the aliases
    filtered_sentences = [sentence for sentence in sentences if any(keyword.lower() in sentence.lower() for keyword in aliases)]
    # Join the filtered sentences back into a single string
    filtered_text = ' '.join(filtered_sentences)
    return filtered_text



features = ['IstALIAS', 'propAN', 'txtSZ', 'countALI', 'countDTS', 'countHOUR', 'countCAPS']

data = pd.read_parquet("data02.parquet")

sonae = data.iloc[3,0].copy()
data.iloc[3,0] = ["Sonae"] ############## ele vai buscar SON mesmo q seja son...

probs_distribution = [] #### probs_distri

for row in data.index:
    print(f"\n {row}", end = ": ")
    aliases = data.loc[row, "aliases"]
    for column in data.columns[1:]:
        print(column, end = " | ")
        validation = []
        for req in data.loc[row, column]:
            text = req["ExtractedText"]
            df = {}
            for feature in features:
                df[feature] = [globals()[feature](text, aliases)]
            #prediction = clf.predict(pd.DataFrame(df)) # binario
            probability = clf.predict_proba(pd.DataFrame(df))[0, 1]
            probs_distribution.append(round(probability, 3)) #### probs_distri
            if probability < 0.4:
                pass
            elif probability >= 0.4 and probability <= 0.6:
                req["newsProbability"] = round(probability, 3)
                req["ExtractedText"] = filter_sentences_by_keywords(text, aliases)
                validation.append(req)
            elif probability > 0.6:
                req["newsProbability"] = round(probability, 3)
                validation.append(req)
        data.loc[row, column] = validation

data.iloc[3,0] = sonae

data.to_parquet("data03.parquet")


 Banco Comercial Português: api.2000 | api.2003 | api.2006 | api.2009 | api.2012 | api.2015 | api.2018 | 
 Galp Energia: api.2000 | api.2003 | api.2006 | api.2009 | api.2012 | api.2015 | api.2018 | 
 EDP: api.2000 | api.2003 | api.2006 | api.2009 | api.2012 | api.2015 | api.2018 | 
 Sonae: api.2000 | api.2003 | api.2006 | api.2009 | api.2012 | api.2015 | api.2018 | 
 Mota-Engil: api.2000 | api.2003 | api.2006 | api.2009 | api.2012 | api.2015 | api.2018 | 

In [5]:
pd.read_parquet("data03.parquet").map(lambda x: len(x)) - pd.read_parquet("data02.parquet").map(lambda x: len(x))

,aliases,api.2000,api.2003,api.2006,api.2009,api.2012,api.2015,api.2018
companies,,,,,,,,
Banco Comercial Português,0,-77,-157,-155,-350,-677,-832,-441
Galp Energia,0,-61,-190,-172,-321,-629,-681,-437
EDP,0,-69,-195,-122,-362,-779,-1064,-432
Sonae,0,-122,-297,-230,-369,-968,-1071,-725
Mota-Engil,0,-3,-63,-23,-117,-285,-470,-265


In [6]:
pd.read_parquet("data03.parquet").map(lambda x: len(x))

,aliases,api.2000,api.2003,api.2006,api.2009,api.2012,api.2015,api.2018
companies,,,,,,,,
Banco Comercial Português,2,13,34,14,147,306,387,383
Galp Energia,2,5,43,9,148,335,423,375
EDP,3,11,50,18,176,297,464,440
Sonae,2,8,21,9,90,141,329,301
Mota-Engil,2,0,4,3,47,99,126,180


In [7]:
data.iloc[0,2][0]

{'ExtractedText': 'RTP BPI não quer lugar no conselho superior do BCP O BPI não pretende ocupar um lugar no conselho superior do Banco Comercial Português (BCP), na sequência da compra de uma posição qualificada no banco de Jardim Gonçalves, adiantou hoje o presidente da instituição. O BPI investiu perto de 107 milhões de euros na compra de uma posição de 2,65% do capital do BCP, no recente aumento de capital do banco presidido por Jardim Gonçalves.',
 'linkToArchive': 'https://arquivo.pt/wayback/20030422094025/http://www.rtp.pt/index.php?article=53268&visual=6',
 'tstamp': '20030422094025',
 'newsProbability': 0.4}

---